## SFT con TRL

[Link a página del curso](https://huggingface.co/learn/llm-course/en/chapter11/3?fw=pt)

In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [14]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

# Set device
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load dataset
dataset = load_dataset("HuggingFaceTB/smoltalk", "everyday-conversations")

# Configure model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(
    device
)

In [15]:
device

'cuda:0'

Cargo tokenizador

In [16]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)


Vamos a:

- Agregar los tokens al modelo y al tokenizador
- Setupear el chat template

In [17]:
sample_messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that provides concise and accurate answers.",
    },
    {
        "role": "user",
        "content": "What is the capital of France?",
    },
    {
        "role": "assistant",
        "content": "The capital of France is Paris.",
    },
]



In [18]:
token2id = tokenizer.get_vocab()
id2token = {v: k for k, v in token2id.items()}

first_tokens = [id2token[i] for i in range(20)]

first_tokens

['<|endoftext|>',
 '<|im_start|>',
 '<|im_end|>',
 '<repo_name>',
 '<reponame>',
 '<file_sep>',
 '<filename>',
 '<gh_stars>',
 '<issue_start>',
 '<issue_comment>',
 '<issue_closed>',
 '<jupyter_start>',
 '<jupyter_text>',
 '<jupyter_code>',
 '<jupyter_output>',
 '<jupyter_script>',
 '<empty_output>',
 '!',
 '"',
 '#']

Ok, los tokens están ahí..

In [19]:
setup_chat_format?

Signature:
setup_chat_format(
    model: transformers.modeling_utils.PreTrainedModel,
    tokenizer: transformers.tokenization_utils.PreTrainedTokenizer,
    format: Optional[Literal['chatml']] = 'chatml',
    resize_to_multiple_of: Optional[int] = None,
) -> tuple[transformers.modeling_utils.PreTrainedModel, transformers.tokenization_utils.PreTrainedTokenizer]
Docstring:
Setup chat format by adding special tokens to the tokenizer, setting the correct format, and extending the embedding layer of the model based on the new special tokens.

If the model already has a chat template, this will throw an error. If you want to overwrite it, please set `tokenizer.chat_template` to `None`.

Args:
    model (`~transformers.PreTrainedModel`): The model to be modified.
    tokenizer (`~transformers.PreTrainedTokenizer`): The tokenizer to be modified.
    format (`Optional[Literal["chatml"]]`): The format to be set. Defaults to "chatml".
    resize_to_multiple_of (`int` or `None`): Number to resize

In [20]:
len(tokenizer.get_vocab())

49152

In [21]:
model, tokenizer = setup_chat_format(model, tokenizer)

In [23]:
len(tokenizer.get_vocab())

49152

In [24]:
print(tokenizer.apply_chat_template(sample_messages, tokenize=False))

<|im_start|>system
You are a helpful assistant that provides concise and accurate answers.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant
The capital of France is Paris.<|im_end|>



## Probemos qué tal anda el modelo base...

In [25]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


In [26]:

# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    bf16=True,
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    processing_class=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [27]:
trainer.train()

RuntimeError: The size of tensor a (8) must match the size of tensor b (16) at non-singleton dimension 0

In [21]:

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
if getattr(tokenizer, "chat_template", None):
    tokenizer.chat_template = None
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    packing=True,
    max_steps=4000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=100,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    eval_steps=500,  # Frequency of evaluation
    bf16=True,
    hub_model_id=finetune_name,  # Set a unique name for your model
    report_to="none",
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    processing_class=tokenizer,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
)

trainer.train()

Applying chat template to train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


RuntimeError: The size of tensor a (4) must match the size of tensor b (8) at non-singleton dimension 0